In [1]:
import datetime
import sys

sys.path.append('../src/')

import fitbit
import gringotts
import config

from bokeh.plotting import figure 
from bokeh.io import output_notebook, show, output_file
from bokeh.models import ColumnDataSource
from bokeh.transform import factor_cmap
from bokeh.palettes import Spectral6

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
def get_client(user_id):
    """ Wrapper for getting a user's client given their id"""

    user_token = gringotts.load_token(user_id)
    
    return fitbit.Fitbit(
        client_id = config.JOE_ID,
        client_secret = config.JOE_SHH,
        access_token = user_token['access_token'],
        refresh_token = user_token['refresh_token'],
        expires_at = user_token['expires_at'],
        refresh_cb = gringotts.save_token,
    )   

In [4]:
ruben_client = get_client('73DJXK')
hasan_client = get_client('6V6CYV')
ori_client = get_client('7954BS')
rosely_client = get_client('7X6MQ9')

In [16]:
# Need to get the date of the start of the week in %Y-%m-%d
today = datetime.date.today()
last_monday = today - datetime.timedelta(days=today.weekday())

In [6]:
today = datetime.datetime(2020, 3, 1)
last_monday = datetime.datetime(2020, 2, 25)#'2020-02-24'

In [17]:
ruben_data = ruben_client.time_series(
    'sleep',
    base_date=last_monday.strftime('%Y-%m-%d'), 
    end_date=today.strftime('%Y-%m-%d')
)
hasan_data = hasan_client.time_series(
    'sleep',
    base_date=last_monday.strftime('%Y-%m-%d'), 
    end_date=today.strftime('%Y-%m-%d')
)
ori_data = ori_client.time_series(
    'sleep',
    base_date=last_monday.strftime('%Y-%m-%d'), 
    end_date=today.strftime('%Y-%m-%d')
)
rosely_data = rosely_client.time_series(
    'sleep',
    base_date=last_monday.strftime('%Y-%m-%d'), 
    end_date=today.strftime('%Y-%m-%d')
)

In [57]:
def get_total_sleep_type(sleep_type, data_dict):
    """ Get the minutes 
    
    Arguments:
        summary_type (str): deep | light | rem | wake
        data_dict (dict): As returned by a client.time_series() call
    """

    total_quantity = 0

    # Iterate over list of dicts where each dict is a night of sleep
    for sleep_dict in data_dict['sleep']:
        # Need more than three hours of sleep to get details about stages
        # Otherwise keys are `asleep`, `awake`, `restless`
        if sleep_type in sleep_dict['levels']['summary']:
#             print(sleep_dict['levels']['summary'])
            total_quantity += sleep_dict['levels']['summary'][sleep_type]['minutes']
        
    return total_quantity


In [58]:
ruben_deep = get_total_sleep_type('deep', ruben_data)
hasan_deep = get_total_sleep_type('deep', hasan_data)
ori_deep = get_total_sleep_type('deep', ori_data)
rosely_deep = get_total_sleep_type('deep', rosely_data)

In [36]:
print(ruben_deep)
print(hasan_deep)
print(ori_deep)
print(rosely_deep)

591
752
197
508


In [30]:
sleep_types = ['deep', 'light', 'rem', 'awake']
users = ['hasan', 'ruben', 'ori', 'rosely']
sleep_type_quantity = [hasan_deep, ruben_deep, ori_deep, rosely_deep]

In [55]:
from bokeh.models import HoverTool

def graph_sleep(users, sleep_type_quantity, sleep_type):
    """ Graph sleep quantities of users
    
    users ([str]): List of user names
    sleep_type_quantity ([int]): List of quantities of sleep.
        Order corresponds to order in users.
    sleep_type: What kind of sleep. deep | light | rem | awake
    """

    source = ColumnDataSource(data=dict(users=users, sleep_type_quantity=sleep_type_quantity))

    p = figure(
        x_range=users,
        plot_height=250,
#         toolbar_location=None,
        title="{sleep_type} Sleep".format(sleep_type=sleep_type.title())
    )
    
    p.vbar(x='users', top='sleep_type_quantity', width=0.5, source=source,
           line_color='white', fill_color=factor_cmap('users', palette=Spectral6, factors=users))

    p.xgrid.grid_line_color = None


    p.add_tools(HoverTool(tooltips=[(f"Total {sleep_type.title()} Sleep", "@sleep_type_quantity")]))

    show(p)

In [56]:
graph_sleep(users, sleep_type_quantity, 'light')